<a href="https://colab.research.google.com/github/DOL-DOM/2022_cau_oss_hackathon/blob/main/hackathon_team10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임 -> 런타임 연결 해제 및 삭제"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
 *    "런타임 -> 런타임 다시시작"은 클라우드 스토리지에 저장된 모델은 유지됨
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2022_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드가 적발될 경우
*  Pre-trained 모델을 사용한 경우 (transfer learning 포함)
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from tensorflow import keras
from keras.utils import np_utils
from keras import datasets, layers, models

# 데이터셋 로드 (MNIST, fashion-MNIST, Kujushiji-MNIST, MNIST_corrupted (test only))
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], shuffle_files=False, batch_size=-1)

train_ds2, test_ds2 = tfds.load('fashion_mnist', split=['train', 'test'], shuffle_files=False, batch_size=-1)
train_ds2['label'] += 10;
test_ds2['label'] += 10;

train_ds3, test_ds3 = tfds.load('kmnist', split=['train', 'test'], shuffle_files=False, batch_size=-1)
train_ds3['label'] += 20;
test_ds3['label'] += 20;

test_ds4 = tfds.load('mnist_corrupted/zigzag', split='test', shuffle_files=False, batch_size=-1)

# 데이터셋 병합 (training: 180,000개, test: 40,000개)
x_train = np.append(np.append(train_ds['image'], train_ds2['image'], 0), train_ds3['image'], 0);
y_train = np.append(np.append(train_ds['label'], train_ds2['label'], 0), train_ds3['label'], 0);

x_test = np.append(np.append(np.append(test_ds['image'], test_ds2['image'], 0), test_ds3['image'], 0), test_ds4['image'], 0);
y_test = np.append(np.append(np.append(test_ds['label'], test_ds2['label'], 0), test_ds3['label'], 0), test_ds4['label'], 0);

# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수: 30, 입력 데이터 구조: (28, 28, 1)
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]
print(num_classes, input_shape)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/mnist_corrupted/zigzag/1.0.0.incomplete8EE6DL/mnist_corrupted-train.tfrecord*.…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/mnist_corrupted/zigzag/1.0.0.incomplete8EE6DL/mnist_corrupted-test.tfrecord*..…

Dataset mnist_corrupted downloaded and prepared to ~/tensorflow_datasets/mnist_corrupted/zigzag/1.0.0. Subsequent calls will reuse this data.
30 (28, 28, 1)


# **2. 데이터 전처리**



In [3]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test_after를 변경하지 말 것
x_train_after = x_train / 255
x_test_after = x_test / 255

# **3. 모델 생성**

In [9]:


drop_rate = 0.3
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Flatten layer: 28 x 28 x 1 image를 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=input_shape))

# 1st hidden layer: fully-connected layer
model.add(keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(keras.layers.Dropout(drop_rate))

# 2nd hidden layer: fully-connected layer 
model.add(keras.layers.Dense(512, activation=tf.nn.relu))
model.add(keras.layers.Dropout(drop_rate))

# 3rd hidden layer: fully-connected layer 
model.add(keras.layers.Dense(256, activation=tf.nn.relu))
model.add(keras.layers.Dropout(drop_rate))

# 4th hidden layer: fully-connected layer 
model.add(keras.layers.Dense(128, activation=tf.nn.relu))
model.add(keras.layers.Dropout(drop_rate))



# Output layer: fully-connected layer 
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')



# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set (x_test1_after, x_test2_after, 혹은 둘을 merge해서 사용)
model.fit(x_train_after, y_train, batch_size = 128, epochs = 100, shuffle=True, callbacks=[cp_callback], validation_data=(x_test_after, y_test))

model.summary()


Epoch 1/100
1397/1407 [============================>.] - ETA: 0s - loss: 0.6688 - accuracy: 0.8004
Epoch 1: val_accuracy improved from -inf to 0.80895, saving model to /content/checkpoint_entire_best.h5
1407/1407 [==============================] - 6s 4ms/step - loss: 0.6672 - accuracy: 0.8008 - val_loss: 0.6279 - val_accuracy: 0.8090
Epoch 2/100
1400/1407 [============================>.] - ETA: 0s - loss: 0.3766 - accuracy: 0.8809
Epoch 2: val_accuracy improved from 0.80895 to 0.81255, saving model to /content/checkpoint_entire_best.h5
1407/1407 [==============================] - 6s 4ms/step - loss: 0.3766 - accuracy: 0.8809 - val_loss: 0.6401 - val_accuracy: 0.8126
Epoch 3/100
1406/1407 [============================>.] - ETA: 0s - loss: 0.3197 - accuracy: 0.8970
Epoch 3: val_accuracy improved from 0.81255 to 0.82028, saving model to /content/checkpoint_entire_best.h5
1407/1407 [==============================] - 7s 5ms/step - loss: 0.3197 - accuracy: 0.8970 - val_loss: 0.6280 - val_acc

KeyboardInterrupt: ignored

# **4. 모델 저장**

In [5]:
save_path = '/content/'
team_name = 'team10'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [6]:
save_path = '/content/'
team_name = 'team10'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test_after, y_test)

1250/1250 [==============================] - 4s 3ms/step - loss: 0.7518 - accuracy: 0.8617


[0.7517508268356323, 0.8616750240325928]